In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import gym

import gym_Snake

import keras as kr

2022-05-12 15:02:29.713873: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-05-12 15:02:29.713894: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
/home/jonas/.local/share/virtualenvs/Tests-qIVfeJrl/lib/python3.8/site-packages/flatbuffers/compat.py:19: DeprecationWarning: the imp module is deprecated in favour of importlib; see the module's documentation for alternative uses
  import imp
/home/jonas/.local/share/virtualenvs/Tests-qIVfeJrl/lib/python3.8/site-packages/keras_preprocessing/image/utils.py:23: DeprecationWarning: NEAREST is deprecated and will be removed in Pillow 10 (2023-07-01). Use Resampling.NEAREST or Dither.NONE instead.
  'nearest': pil_image.NEAREST,
/home/jonas/.local/share/virtualenvs/Tests-qIVfeJrl/lib/python3.8/site-packages/k

In [2]:
#input_shape = (obs.shape)
input_shape = (10,10,1)

output = 3


# Classic 3L CNN into 2 dense output layer serving as Deep Q Network
# 3 conv2D layer into maxpooling into flatten into 2 denses layers

# Can remove the max pooling maybe, and change last layer softmax -> linear if simpler needed
model = kr.models.Sequential(
    [
        kr.layers.Input(shape=input_shape),
        kr.layers.Conv2D(32, kernel_size=(3, 3),activation="relu"),
        kr.layers.Conv2D(64, kernel_size=(3, 3),activation="relu"),
        kr.layers.Conv2D(128, kernel_size=(3, 3),activation="relu"),
        kr.layers.MaxPooling2D(pool_size=(2,2)),
        kr.layers.Flatten(),
        kr.layers.Dense(256,activation="relu"),
        kr.layers.Dense(output,activation="softmax"),
        
    ]
)

model.compile(
    loss='mse',
    optimizer='adam',
    metrics=['accuracy']
)

2022-05-12 15:02:30.703131: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2022-05-12 15:02:30.703149: W tensorflow/stream_executor/cuda/cuda_driver.cc:269] failed call to cuInit: UNKNOWN ERROR (303)
2022-05-12 15:02:30.703168: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (jonasAspire): /proc/driver/nvidia/version does not exist
2022-05-12 15:02:30.703389: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [3]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 8, 8, 32)          320       
                                                                 
 conv2d_1 (Conv2D)           (None, 6, 6, 64)          18496     
                                                                 
 conv2d_2 (Conv2D)           (None, 4, 4, 128)         73856     
                                                                 
 max_pooling2d (MaxPooling2D  (None, 2, 2, 128)        0         
 )                                                               
                                                                 
 flatten (Flatten)           (None, 512)               0         
                                                                 
 dense (Dense)               (None, 256)               131328    
                                                        

In [4]:
env = gym.make('Snake-v0', 
               player='computer', 
               shape='Normal', 
               state_mode='matrix', 
               reward_mode = 'extended', 
               width=10, 
               height=10, 
               solid_border=True)
print(env.observation_space)
print(env.action_space)



Discrete(100)
Discrete(3)


In [5]:
obs = env.reset()
done = False

i = 0

while not done:
    # Render
    env.render()
    
    # Get network prediction
    pred = model.predict(obs.reshape((1,10,10,1)))
    
    # Get maximal value
    choice = np.argmax(pred)

    # Act in the env
    obs, reward, done, info = env.step(choice)
    
    # Train model. But how?
    # Dummy learning, just teach to take second action
    model.fit([obs.reshape((1,10,10,1))], np.array([[0,1,0]]))
    
    i += 1
    if i > 25: break
#     model.train()

[[0.31607556 0.30083075 0.38309366]]
<class 'numpy.ndarray'>
1/1 [==============================] - 12s 12s/step - loss: 0.2769 - accuracy: 0.0000e+00
[[0.21434122 0.5318079  0.25385094]]
<class 'numpy.ndarray'>
1/1 [==============================] - 0s 78ms/step - loss: 0.1606 - accuracy: 1.0000
[[0.15568827 0.6704507  0.17386109]]
<class 'numpy.ndarray'>
1/1 [==============================] - 0s 69ms/step - loss: 0.1004 - accuracy: 1.0000
[[0.14158154 0.712151   0.1462675 ]]
<class 'numpy.ndarray'>
1/1 [==============================] - 0s 67ms/step - loss: 0.0289 - accuracy: 1.0000
[[0.05235125 0.8918982  0.05575052]]
<class 'numpy.ndarray'>
1/1 [==============================] - 0s 42ms/step - loss: 0.0018 - accuracy: 1.0000
[[0.00985242 0.97884333 0.01130426]]
<class 'numpy.ndarray'>
1/1 [==============================] - 0s 83ms/step - loss: 3.3337e-04 - accuracy: 1.0000
[[0.00418831 0.99117166 0.00464   ]]
<class 'numpy.ndarray'>
1/1 [==============================] - 0s 78ms/st